In [9]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from scipy.stats import entropy
import numpy as np
import gc
import shapely
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

tqdm.pandas()

part = 6

bu = gpd.read_parquet(f'data/buurten_parts/buurten_c_{part}.parquet')

categorisation = pd.read_excel('data/categorisation.xlsx')
L0_cats = categorisation['L0 category'].unique()
L1_cats = categorisation['L1 category'].unique()

In [10]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

In [11]:
bu.head(3)

,buurtcode,buurtnaam,wijkcode,gemeentecode,gemeentenaam,IND_WBI,H2O,POSTCODE,DEK_PERC,OAD,...,total_amenities_1,L0_leibovici_1,L1_leibovici_1,L0_altieri_1,L1_altieri_1,total_amenities_2,L0_leibovici_2,L1_leibovici_2,L0_altieri_2,L1_altieri_2
8367,BU08280305,Oranjebuurt II,WK082803,GM0828,Oss,1.0,NEE,5342,1.0,1113.0,...,1.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,NaN
8368,BU08280306,Willibrordusweg-Oost,WK082803,GM0828,Oss,1.0,NEE,5342,1.0,542.0,...,1.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,NaN
8369,BU08280307,Willibrordusweg-West,WK082803,GM0828,Oss,1.0,NEE,5342,1.0,701.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
for idx, row in tqdm(bu.iterrows(), total=bu.shape[0]):
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{row['gemeentenaam']}.parquet")
    amenities = amenities[amenities.geometry.within(row.geometry)]
    for filter_i in [0, 1, 2]:
        
        # get and apply filter
        L0_filter, L1_filter = ec.getfilter(filter_i)
        
        amenities_f = amenities[~amenities.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_f = amenities_f[
                    ~( (amenities_f.L0_category == key) & (amenities_f.L1_category.isin(value)) )
                ]
        
        bu.at[idx, f'L0_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L0_category.values)
        bu.at[idx, f'L1_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L1_category.values)
        
        for cat in L0_cats:
            bu.loc[idx, f'L1_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L0_category == cat])
        
        for cat in L1_cats:
            bu.loc[idx, f'L0_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L1_category == cat])

    del amenities
    del amenities_f
    gc.collect()

bu.to_parquet(f'data/buurten_parts/calculated/buurten_c_{part}.parquet') 

100%|██████████| 1394/1394 [20:38<00:00,  1.13it/s]
